# CNN for one subject

In [4]:
%pylab
%matplotlib inline

import glob
import os
import mne
CORPORA_PATH = "~/projects/corpora/P3Speller/P3Speller-old-y-datos/sets/"

file_path = os.path.expanduser(CORPORA_PATH)
files = glob.glob(os.path.join(file_path, "*.set"))

def normalize_subject(X):
    mean = X.mean(axis=(0, 2)).reshape(-1, 1)
    std = X.std(axis=(0, 2)).reshape(-1, 1)
    return (X - mean) / std

def load_data(filename, normalize=True):
    data_mne = mne.io.read_raw_eeglab(filename, preload=True, event_id={"0": 1, "1": 2})
    data_mne.filter(0, 20)
    events = mne.find_events(data_mne)
    epochs = mne.Epochs(
        data_mne, events,
        baseline=(None, 0), tmin=-0.1, tmax=0.7)

    epochs.load_data()
    
    ch_names = epochs.ch_names
    
    X = epochs.get_data()[:, :-1]
    y = (events[:, 2] == 2).astype('float')

    if normalize: 
        X = normalize_subject(X)
    
    return X, y 

filename = files[109]

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


Targets appear as 2 in the third column


We remove last channel as well

In [5]:
import sklearn.utils

X, y = load_data(filename)


Reading /home/jmperez/projects/corpora/P3Speller/P3Speller-old-y-datos/sets/PruebasMuseo_30261001.fdt
Reading 0 ... 94591  =      0.000 ...   738.992 secs...
Setting up low-pass filter at 20 Hz
h_trans_bandwidth chosen to be 5.0 Hz
Filter length of 169 samples (1.320 sec) selected
2700 events found
Events id: [1 2]
2700 matching events found
0 projection items activated
Loading data for 2700 events and 104 original time points ...
0 bad epochs dropped


In [6]:
n = y.shape[0]

print("Balance Class 0 : {} Class 1 {}".format(sum(y == 0)/n, sum(y==1) / n))

Balance Class 0 : 0.8333333333333334 Class 1 0.16666666666666666


In [7]:
X.shape, y.shape

((2700, 14, 104), (2700,))

In [8]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.1, stratify=y)

# Simple MLP 

In [9]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPool2D, Flatten, Dense, Dropout


model = Sequential()

input_shape = X.shape[1:]

#n_kernels = 100
#model.add(Conv1D(n_kernels, 10, 
#                activation='sigmoid', input_shape=(14, 104)))
model.add(Flatten(input_shape=(14, 104)))
model.add(Dropout(0.55))
model.add(Dense(4096, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mse', # using the cross-entropy loss function
              optimizer='adam', 
              metrics=['accuracy']) # reporting the accuracy

Using TensorFlow backend.
/home/jmperez/.pyenv/versions/3.6.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [10]:
model.fit(X_train, y_train, epochs=40, batch_size=64, class_weight={0:1, 1:6})

Epoch 1/40
2430/2430 [==============================] - 3s 1ms/step - loss: 0.7408 - acc: 0.4704
Epoch 2/40
2430/2430 [==============================] - 3s 1ms/step - loss: 0.6776 - acc: 0.4687
Epoch 3/40
2430/2430 [==============================] - 3s 1ms/step - loss: 0.7196 - acc: 0.4181
Epoch 4/40
2430/2430 [==============================] - 3s 1ms/step - loss: 0.6646 - acc: 0.4930
Epoch 5/40
2430/2430 [==============================] - 3s 1ms/step - loss: 0.6651 - acc: 0.4263
Epoch 6/40
2430/2430 [==============================] - 3s 1ms/step - loss: 0.6435 - acc: 0.5033
Epoch 7/40
2430/2430 [==============================] - 3s 1ms/step - loss: 0.6852 - acc: 0.4218
Epoch 8/40
2430/2430 [==============================] - 3s 1ms/step - loss: 0.6229 - acc: 0.5185
Epoch 9/40
2430/2430 [==============================] - 3s 1ms/step - loss: 0.5657 - acc: 0.5041
Epoch 10/40
2430/2430 [==============================] - 3s 1ms/step - loss: 0.5096 - acc: 0.5539
Epoch 11/40
2430/2430 [======

In [11]:
model.evaluate(X_test, y_test, verbose=1)


270/270 [==============================] - 0s 464us/step


[0.17590948321201183, 0.77037037125340213]

In [12]:
y_pred = model.predict_classes(X_test)
y_prob = model.predict(X_test)
#np.hstack((y_prob, y_pred, y_test.reshape(-1, 1)))

In [14]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)

print("""
Accuracy   = {}
Precision  = {}
Recall     = {}
ROC AUC    = {}
""".format(accuracy, precision, recall, auc))


Accuracy   = 0.7703703703703704
Precision  = 0.24242424242424243
Recall     = 0.17777777777777778
ROC AUC    = 0.5885432098765432



# CNN

As we consider the last dimension the channels, we should invert the matrix...

In [32]:
X.shape

(2700, 14, 104)

In [33]:
X_t = np.transpose(X, (0,2,1))


In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X_t, y, test_size=0.1, stratify=y)

In [58]:

model = Sequential()

input_shape = X.shape[1:]

n_kernels = 10
model.add(Conv1D(n_kernels, 10, padding='same', 
                activation='relu', input_shape=(104, 14)))
model.add(Conv1D(n_kernels, 20, padding='same', 
                activation='relu'))
model.add(Flatten(input_shape=(14, 104)))
model.add(Dropout(0.55))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # using the cross-entropy loss function
              optimizer='rmsprop', 
              metrics=['accuracy']) # reporting the accuracy

In [59]:
model.fit(X_train, y_train, epochs=20, batch_size=64, class_weight={0:1, 1:6})

Epoch 1/20
2430/2430 [==============================] - 1s 376us/step - loss: 1.4220 - acc: 0.4119
Epoch 2/20
2430/2430 [==============================] - 1s 243us/step - loss: 1.2748 - acc: 0.4457
Epoch 3/20
2430/2430 [==============================] - 1s 248us/step - loss: 1.2427 - acc: 0.4539
Epoch 4/20
2430/2430 [==============================] - 1s 251us/step - loss: 1.2169 - acc: 0.4971
Epoch 5/20
2430/2430 [==============================] - 1s 239us/step - loss: 1.1694 - acc: 0.5560
Epoch 6/20
2430/2430 [==============================] - 1s 251us/step - loss: 1.1308 - acc: 0.5984
Epoch 7/20
2430/2430 [==============================] - 1s 242us/step - loss: 1.1073 - acc: 0.6037
Epoch 8/20
2430/2430 [==============================] - 1s 245us/step - loss: 1.0431 - acc: 0.6436
Epoch 9/20
2430/2430 [==============================] - 1s 255us/step - loss: 1.0586 - acc: 0.6494
Epoch 10/20
2430/2430 [==============================] - 1s 250us/step - loss: 1.0303 - acc: 0.6613
Epoch 11/

In [60]:
y_pred = model.predict_classes(X_test)
y_prob = model.predict(X_test)
#np.hstack((y_prob, y_pred, y_test.reshape(-1, 1)))

from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)

print("""
Accuracy   = {}
Precision  = {}
Recall     = {}
ROC AUC    = {}
""".format(accuracy, precision, recall, auc))


Accuracy   = 0.6666666666666666
Precision  = 0.1917808219178082
Recall     = 0.3111111111111111
ROC AUC    = 0.5585185185185185




## Second CNN architecture

```
The network topology is the key feature in the classifier. 
The network is composed of five layers, which are themselves
composed of one or several maps. A map represents a layer entity, 
which has a specific semantic: Each map of the first hidden layer 
is a channel combination. The second hidden layer subsamples and 
transforms the signal in the time  domain.

The classifier architecture is presented in Fig. 3. The
number of neurons for each map is presented between
brackets; the size of the convolution kernel is between hooks.

The order of the convolution is chosen in relation to what is
traditionally done in BCI. First, optimal spatial filters/
channel combinations are set, then the signal is processed in
the time domain. 

The choice of the topology is also justified by
the possibility of easily interpreting the trained convolution
kernel, i.e., the receptive fields. In the proposed strategy, the
kernels are vectors and not matrix, like in CNNs for image
recognition. The reason is to not mix in one kernel features
related to the space and time domain.
The network topology is described as follows:
```

As we are going to use 2d kernels, we add a Dummy Dimension (as last channel)

In [80]:
X_t = X[:, :, :, np.newaxis]

X_t.shape

(2700, 14, 104, 1)

In [83]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X_t, y, test_size=0.1, stratify=y)

X_train.shape, y_train.shape

((2430, 14, 104, 1), (2430,))

In [99]:
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Flatten, Dense, Dropout

model = Sequential()

n_kernels = 10
model.add(Conv2D(n_kernels, (14, 1), padding='same', 
                activation='relu', input_shape=(14, 104, 1)))
model.add(Conv2D(5*n_kernels, (1, 13), padding='same',
                activation='relu'))
model.add(Flatten())
model.add(Dropout(0.55))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # using the cross-entropy loss function
              optimizer='adam', 
              metrics=['accuracy']) # reporting the accuracy


In [106]:
model.fit(X_train, y_train, epochs=10, batch_size=64, class_weight={0:1, 1:6})

Epoch 1/10
2430/2430 [==============================] - 10s 4ms/step - loss: 0.0143 - acc: 0.9988
Epoch 2/10
2430/2430 [==============================] - 10s 4ms/step - loss: 0.0127 - acc: 0.9984
Epoch 3/10
2430/2430 [==============================] - 10s 4ms/step - loss: 0.0087 - acc: 1.0000
Epoch 4/10
2430/2430 [==============================] - 10s 4ms/step - loss: 0.0081 - acc: 0.9996
Epoch 5/10
2430/2430 [==============================] - 11s 4ms/step - loss: 0.0059 - acc: 1.0000
Epoch 6/10
2430/2430 [==============================] - 10s 4ms/step - loss: 0.0047 - acc: 1.0000
Epoch 7/10
2430/2430 [==============================] - 12s 5ms/step - loss: 0.0049 - acc: 1.0000
Epoch 8/10
2430/2430 [==============================] - 12s 5ms/step - loss: 0.0052 - acc: 0.9996
Epoch 9/10
2430/2430 [==============================] - 12s 5ms/step - loss: 0.0076 - acc: 1.0000
Epoch 10/10
2430/2430 [==============================] - 12s 5ms/step - loss: 0.0140 - acc: 0.9971


In [105]:
y_pred = model.predict_classes(X_test)
y_prob = model.predict(X_test)
#np.hstack((y_prob, y_pred, y_test.reshape(-1, 1)))

from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)

print("""
Accuracy   = {}
Precision  = {}
Recall     = {}
ROC AUC    = {}
""".format(accuracy, precision, recall, auc))


Accuracy   = 0.7814814814814814
Precision  = 0.15
Recall     = 0.06666666666666667
ROC AUC    = 0.5723456790123457

